#### Deliverable 6: Sales Fact Table (Daily Level) Implementation

In [21]:
import pandas as pd
import sqlite3 as lite

In [22]:
conn = lite.connect("store1.db")
output_file_path = "./output/"

In [23]:
# load dimension tables
date_dim = pd.read_sql("select DateKey, Date from DateDimension", conn)
products = pd.read_sql("select ProductKey, SKU from ProductDimension", conn)
stores = pd.read_sql("select StoreKey from StoreDimension", conn)

In [24]:
store_databases = {
    "Store 1": {
        "database_name": "store1.db",
        "transactions_table_name": "sales_transactions",
    },
    "Store 2": {
        "database_name": "store2.db",
        "transactions_table_name": "sales_transactions",
    },
    "Store 3": {
        "database_name": "store3.db",
        "transactions_table_name": "sales_transactions",
    },
    "Store 4": {
        "database_name": "store4.db",
        "transactions_table_name": "sales_transactions"
    }
}

In [25]:
def load_all_transactions(store):
    conn = lite.connect(store['database_name'])
    table_name = store['transactions_table_name']

    transactions_query = f"select * from {table_name}"
    transaction_df = pd.read_sql(transactions_query, conn)

    conn.close()
    return transaction_df

In [26]:
load_all_transactions(store_databases["Store 1"]).head()

,date,customer_number,sku,salesPrice,items_left,cases_ordered
0,2024-01-01,1,42356001,2.08,191,16
1,2024-01-01,1,44037001,0.87,23,2
2,2024-01-01,1,43029001,12.09,23,2
3,2024-01-01,1,43247001,7.69,83,7
4,2024-01-01,1,43766001,3.72,83,7


In [27]:
all_daily_transactions = []

def new_func(transactions):
    daily_agg = transactions.groupby(['DateKey', "ProductKey", "StoreKey"]).agg({
        'QuantitySold': 'sum',
        'TotalDollarSales': 'sum',
        'TotalCostToStore': 'sum',
        'GrossProfit': 'sum',
        'customer_number': 'nunique'   # Count unique customers per day
    }).reset_index()
    
    # Format TotalDollarSales to 2 decimal points
    daily_agg['TotalDollarSales'] = daily_agg['TotalDollarSales'].round(2)
    daily_agg['TotalCostToStore'] = daily_agg['TotalCostToStore'].round(2)
    daily_agg['GrossProfit'] = daily_agg['GrossProfit'].round(2)
    
    return daily_agg

for store_name, config in store_databases.items():
    print(f"Processing {store_name}...")

    # Load transactions
    transactions = load_all_transactions(config)
    transactions['StoreKey'] = store_name.split(" ")[1]

    # Convert date to datetime and merge with date dimension
    transactions['temp_date'] = pd.to_datetime(transactions['date'])
    transactions = transactions.merge(
        date_dim,
        left_on="temp_date",
        right_on=pd.to_datetime(date_dim['Date']),
        how='left'
    ).drop(columns=['temp_date', 'Date'])

    # Ensure 'sku' is the same type as 'SKU'
    transactions['SKU'] = pd.to_numeric(
        # Handles NaN if needed
        transactions['sku'], errors='coerce').astype('Int64')

    # Merge with product dimension
    transactions = transactions.merge(
        products,
        left_on='sku',
        right_on='SKU',
        how="left"
    )

    # Calculate metrics
    transactions['QuantitySold'] = transactions['cases_ordered'] * 12
    transactions['TotalDollarSales'] = transactions['salesPrice'] * \
        transactions['QuantitySold']

    # Assume the store buys the product at 70% of the retail price
    transactions['TotalCostToStore'] = transactions['salesPrice'] * \
        0.7 * transactions['QuantitySold']

    transactions['GrossProfit'] = transactions['TotalDollarSales'] - \
        transactions['TotalCostToStore']

    # Group by DateKey, ProductKey, StoreKey to get daily aggregates
    daily_agg = new_func(transactions)

    # Rename columns to match the required schema
    daily_agg = daily_agg.rename(columns={
        'QuantitySold': '#SoldToday',
        'TotalDollarSales': 'SalesTotal',
        'TotalCostToStore': 'CostOfItemsSold',
        'customer_number': 'DailyCustomerCount'
    })

    all_daily_transactions.append(daily_agg)

Processing Store 1...
Processing Store 2...
Processing Store 3...
Processing Store 4...


In [28]:
# combine all stores data
daily_sales_fact = pd.concat(all_daily_transactions)

In [29]:
daily_sales_fact.sample(5)

,DateKey,ProductKey,StoreKey,#SoldToday,SalesTotal,CostOfItemsSold,GrossProfit,DailyCustomerCount
977217,485,741,2,222072,550738.56,385516.99,165221.57,19
1015291,504,547,2,357048,931895.28,652326.70,279568.58,28
513802,256,12,1,63168,69484.80,48639.36,20845.44,14
384009,191,1164,1,62856,483362.64,338353.85,145008.79,18
232303,116,85,3,98952,296856.00,207799.20,89056.80,31


In [30]:
daily_sales_fact.shape

(4415200, 8)

In [31]:
daily_sales_fact['StoreKey'].value_counts()

StoreKey
4    1105095
3    1104495
2    1103498
1    1102112
Name: count, dtype: int64

##### Filter out any rows with missing keys

In [32]:
daily_sales_fact = daily_sales_fact.dropna(subset=['DateKey', "ProductKey", "StoreKey"])
daily_sales_fact.sample(5)

,DateKey,ProductKey,StoreKey,#SoldToday,SalesTotal,CostOfItemsSold,GrossProfit,DailyCustomerCount
130710,65,1525,4,29988,35085.96,24560.17,10525.79,21
192873,96,1439,1,21312,22803.84,15962.69,6841.15,12
186173,93,413,3,86400,228960.00,160272.00,68688.00,31
262108,130,1965,2,59964,97141.68,67999.18,29142.50,19
376744,187,1029,4,70788,223690.08,156583.06,67107.02,17


In [33]:
daily_sales_fact.shape

(4415200, 8)

##### convert keys to integers

In [34]:
daily_sales_fact.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4415200 entries, 0 to 1105094
Data columns (total 8 columns):
 #   Column              Dtype  
---  ------              -----  
 0   DateKey             int64  
 1   ProductKey          int64  
 2   StoreKey            object 
 3   #SoldToday          int64  
 4   SalesTotal          float64
 5   CostOfItemsSold     float64
 6   GrossProfit         float64
 7   DailyCustomerCount  int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 303.2+ MB


In [35]:
daily_sales_fact['DateKey'] = daily_sales_fact['DateKey'].astype(int)
daily_sales_fact['ProductKey'] = daily_sales_fact['ProductKey'].astype(int)
daily_sales_fact['StoreKey'] = daily_sales_fact['StoreKey'].astype(int)

In [36]:
curr = conn.cursor()

curr.execute("Drop table if exists SalesFact_DailyLevel")
curr.execute("""
    CREATE TABLE SalesFact_DailyLevel (
        DateKey INT NOT NULL,
        ProductKey INT NOT NULL,
        StoreKey INT NOT NULL,
        "#SoldToday" INT NOT NULL,
        CostOfItemsSold REAL NOT NULL,
        SalesTotal REAL NOT NULL,
        GrossProfit REAL NOT NULL,
        DailyCustomerCount INT NOT NULL,
        PRIMARY KEY (DateKey, ProductKey, StoreKey),
        FOREIGN KEY (DateKey) REFERENCES DateDimension(DateKey),
        FOREIGN KEY (ProductKey) REFERENCES ProductDimension(ProductKey),
        FOREIGN KEY (StoreKey) REFERENCES StoreDimension(StoreKey)
    );
""")

In [37]:
daily_sales_fact.to_sql("SalesFact_DailyLevel", conn, if_exists='replace', index=False)

4415200

##### Test

In [38]:
sales_fact = pd.read_sql("select * from SalesFact_DailyLevel limit 10", conn)
sales_fact

,DateKey,ProductKey,StoreKey,#SoldToday,SalesTotal,CostOfItemsSold,GrossProfit,DailyCustomerCount
0,1,1,1,1596,4373.04,3061.13,1311.91,19
1,1,2,1,924,1820.28,1274.20,546.08,11
2,1,3,1,1092,4793.88,3355.72,1438.16,12
3,1,4,1,1680,7375.20,5162.64,2212.56,20
4,1,5,1,672,2210.88,1547.62,663.26,8
5,1,6,1,1764,3880.80,2716.56,1164.24,19
6,1,7,1,1680,21420.00,14994.00,6426.00,20
7,1,8,1,2268,12451.32,8715.92,3735.40,27
8,1,9,1,1512,5972.40,4180.68,1791.72,18
9,1,10,1,2352,12395.04,8676.53,3718.51,28


In [39]:
curr.close()
conn.close()

In [40]:
daily_sales_fact.to_csv(f"{output_file_path}SalesFact_DailyLevel.csv", index=False)